## 匯入需要的函式庫

In [1]:
import pandas as pd

# 結巴
import jieba

# 讀取資料用
import glob
import os

# 清理資料
import re
import string

### 讀取檔案所在的資料夾內容

In [2]:
# 輸入分類資料夾的位置
path = './106'

if path[len(path)-1] != '/':
    path = path + '/'
    
# 紀錄分類資料夾裡的資料夾資料
directory = glob.glob(path + '*')
directory

['./106/1',
 './106/2',
 './106/3',
 './106/4',
 './106/5',
 './106/6',
 './106/7',
 './106/all 7 ratings']

### 清理不要的資料夾

In [3]:
directory.remove('./106/all 7 ratings')
directory

['./106/1', './106/2', './106/3', './106/4', './106/5', './106/6', './106/7']

### 列出各資料夾內的資料

In [4]:
file = []
for dir_name in directory:
    print('In folder', dir_name, ':')
    file_in_dir = []
    for file_name in glob.glob(dir_name + '/*'):
        print('\t', file_name)
        file_in_dir.append(file_name)
    file.append(file_in_dir)

In folder ./106/1 :
	 ./106/1/1062890.txt
In folder ./106/2 :
In folder ./106/3 :
In folder ./106/4 :
	 ./106/4/1062836.txt
In folder ./106/5 :
	 ./106/5/1062891.txt
In folder ./106/6 :
	 ./106/6/1062801.txt
	 ./106/6/1062834.txt
	 ./106/6/1062838.txt
	 ./106/6/1062845.txt
	 ./106/6/1062849.txt
	 ./106/6/1062880.txt
	 ./106/6/1062883.txt
	 ./106/6/1062886.txt
	 ./106/6/1062887.txt
	 ./106/6/1062892.txt
	 ./106/6/1065880.txt
In folder ./106/7 :
	 ./106/7/1062809.txt
	 ./106/7/1062881(缺).txt
	 ./106/7/1062882.txt
	 ./106/7/1062884.txt


### 讀取檔案並紀錄其 level, 檔案名稱(title), 檔案內容(documnet)

In [5]:
title = []
level = []
path = []
document = []
for dir_name in file:
    for file_name in dir_name:
        path.append(file_name)
        level.append(file_name.split('/')[-2])
        title.append(os.path.basename(file_name).split('.')[0])
        with open(file_name, 'r', encoding='utf-8') as data:
            document.append(data.read())

for i in range(len(level)):
    print('level :', level[i], ', name :', title[i])

level : 1 , name : 1062890
level : 4 , name : 1062836
level : 5 , name : 1062891
level : 6 , name : 1062801
level : 6 , name : 1062834
level : 6 , name : 1062838
level : 6 , name : 1062845
level : 6 , name : 1062849
level : 6 , name : 1062880
level : 6 , name : 1062883
level : 6 , name : 1062886
level : 6 , name : 1062887
level : 6 , name : 1062892
level : 6 , name : 1065880
level : 7 , name : 1062809
level : 7 , name : 1062881(缺)
level : 7 , name : 1062882
level : 7 , name : 1062884


### 顯示文件部份內容

In [6]:
# 要預覽的文件數量
num_of_print_doc = 10
# 每個文件顯示的長度
num_of_print_len = 1000
# 防呆
if num_of_print_len > len(document):
    num_of_print_len = len(document)

for i in range(num_of_print_doc):
    print('------document: {0}------'.format(title[i]))
    print(document[i][:num_of_print_len], ' ... ', document[i][-num_of_print_len:])

------document: 1062890------
﻿2017年景氣及市場回顧
2017  ...  品牌」，以不負全體股東之殷切寄託。

------document: 1062836------
"回顧106年，美國聯準會升息三次且  ...  理
念，成為本國銀行中的優質銀行。"
------document: 1062891------
﻿壹、致股東報告書
親愛的股東女士先  ...  心目中最值得信賴的
金融服務機構。

------document: 1062801------
﻿"壹│致股東報告書
 張明道
10  ...  
-
穩定
  董事長 總經理
 "
------document: 1062834------
﻿"壹 致股東報告書
05 10 1  ...  BB+
A-2
Stable
11"
------document: 1062838------
"一、106年度營業成果
（一）國內  ...  位股東繼續不吝給予指
導與鼓勵。
"
------document: 1062845------
"各位女士、先生：
回顧 106 年  ...  位金融科技發展，開發線上投保商機。"
------document: 1062849------
"各位股東女士、先生：
回顧去年，全  ...  大價值，建構永續經營的穩固基礎。
"
------document: 1062880------
﻿一、集團 106 年度經營結果
(  ...  t(2018/01) A a 穩定

------document: 1062883------
﻿壹、 致股東報告書
回顧民國(下同  ...  成企業永續經營之目標。
最後敬祝大家


### 清除數字, 符號, 英文, 空白字元
* True : 清除
* False: 不清除

In [7]:
# 移除數字
remove_number = True

# 移除符號
remove_puncs = True

# 移除英文
remove_eng = True

# 移除空白字元, 包含換行; tab; 空白等
remove_space = False

# 選擇移除的字元由什麼取代
# 預設移除字元由空白取代
remove_by = ' '

def remove_special_char():
    # 移除數字
    if(remove_number):
        for i in range(len(document)):
            document[i] = re.sub('\d', remove_by, document[i])
    # 移除符號
    if(remove_puncs):
        puncs_Half = "!?.#$%&'()*+,-/:;<=>@[]^_{}|│~\\\""
        # 太長分兩段寫
        puncs_Full = '！？｡＂＃＄％＆＇（）()＊＋，－／：；＜＝＞＠［＼］$＾＿｀｛｜｝～｟｠｢｣､、〃《》'
        puncs_Full = puncs_Full + '「」『』【】〔〕〖〗〘〙〚〛`〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏.。'
        puncs = puncs_Half + puncs_Full
        for i in range(len(document)):
            text = document[i]
            document[i] = ''
            for word in text:
                if word not in puncs:
                    document[i] = document[i] + word
                else:
                    document[i] = document[i] + remove_by
    # 移除英文
    if(remove_eng):
        for i in range(len(document)):
            document[i] = re.sub('[a-zA-Z]', remove_by, document[i])
    # 移除空白字元
    if(remove_space):
        for i in range(len(document)):
            document[i] = re.sub('\s', '', document[i])
remove_special_char()

### 印出部份結果

In [8]:
# 要預覽的文件數量
num_of_print_doc = 10
# 每個文件顯示的長度
num_of_print_len = 1000
# 防呆
if num_of_print_len > len(document):
    num_of_print_len = len(document)

for i in range(num_of_print_doc):
    print('------document: {0}------'.format(title[i]))
    print(document[i][:num_of_print_len], ' ... ', document[i][-num_of_print_len:])

------document: 1062890------
﻿    年景氣及市場回顧
      ...  品牌  以不負全體股東之殷切寄託 

------document: 1062836------
 回顧   年 美國聯準會升息三次且  ...  理
念 成為本國銀行中的優質銀行  
------document: 1062891------
﻿壹 致股東報告書
親愛的股東女士先  ...  心目中最值得信賴的
金融服務機構 

------document: 1062801------
﻿ 壹 致股東報告書
 張明道
    ...  
 
穩定
  董事長 總經理
  
------document: 1062834------
﻿ 壹 致股東報告書
         ...     
   
      
   
------document: 1062838------
 一    年度營業成果
 一 國內  ...  位股東繼續不吝給予指
導與鼓勵 
 
------document: 1062845------
 各位女士 先生 
回顧     年  ...  位金融科技發展 開發線上投保商機  
------document: 1062849------
 各位股東女士 先生 
回顧去年 全  ...  大價值 建構永續經營的穩固基礎 
 
------document: 1062880------
﻿一 集團     年度經營結果
   ...                 穩定

------document: 1062883------
﻿壹  致股東報告書
回顧民國 下同  ...  成企業永續經營之目標 
最後敬祝大家


---
### 載入結巴字典檔
#### 字典檔格式
* 一組字詞一行
* 每一行可寫入 詞語 詞語權重(可略) 詞性(可略), 以空白分隔

In [9]:
# 輸入字典檔的位置
dictionary_path = './dict.txt'

if os.path.isfile(dictionary_path):
    jieba.load_userdict(file_name)
else:
    print('字典檔不存在，請確認字典檔路徑是否正確！')

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.977 seconds.
Prefix dict has been built succesfully.


### 讀取停止詞字典，並顯示部份內容(前200字)

In [10]:
stopword_path = './stopword.txt'
num_of_word_to_print = 200

if os.path.isfile(stopword_path):
    with open(stopword_path, 'r', encoding='utf-8') as fp:
        stopword = fp.read().split('\n')[:-1]
        print(stopword[:num_of_word_to_print])
else:
    print('停止詞字典不存在，請確認路徑是否正確！')

['因為', '查核', '報告', '性事', '增加', '會計', '相對', '仟元', '金額', '權益', '能力', '利益', '股東', '合理性', '假設', '無形', '上', '開關鍵', '模型', '支出', '控制', '計算', '資金', '評估', '報告', '因應', '仟元', '金額', '權益', '假設', '無形', '上', '模型', '支出', '控制', '計算', '資金', '評估', '人次', '重大', '資訊', '總額', '民國', '估計', '', '一副', '公司', '年度', '包括', '正確性', '正', '確性', '減損', '共為', '預估', '預算', '維持', '公開', '整體', '表示', '特性', '市場', '科目', '發生', '年本', '重要', '判斷', '主要', '流量', '意見', '關鍵', '同期', '評價', '正確', '地區', '總資產', '執行', '程序', '測試', '稅後淨利', '股份有限公司', '三日', '之應', '無法', '百之', '點之', '一季', '一步', '一波', '一環', '一百', '一直', '一經', '一致', '一遭', '一部', '一龍', '三方', '三種', '三線', '三親', '上斷', '上本', '並依', '並列', '並加蓋', '並參酌', '並取', '並合', '並呈', '並將', '並對', '並引', '並應', '並擬', '並斷', '並於', '並於給', '並檢', '並無', '並無給', '並無顯', '並爭', '並申', '並監控', '並確', '並積極', '並經', '並繼', '並與', '並視', '並認', '並重', '中信', '中劃', '中境', '中將', '中房', '中逆勢', '中陸', '之丁', '之上', '之丕', '之丙', '之互', '之五', '之任', '之份', '之供', '之列', '之判斷', '之利', '之半', '之執', '之場', '之壽命', '之將', '之履', '之平', '之年', '之庫', '之廠', '之延', '之延遲',

### 利用結巴及停止詞字典清理資料

In [11]:
for i in range(len(document)):
    text = jieba.cut(document[i], cut_all=False)
    filtered = []
    for word in text:
        if(word not in stopword) and (word!=' '):
            filtered.append(word)
    document[i] = ' '.join(filtered)
    
# 要預覽的文件數量
num_of_print_doc = 10
# 每個文件顯示的長度
num_of_print_len = 1000
# 防呆
if num_of_print_len > len(document):
    num_of_print_len = len(document)

for i in range(num_of_print_doc):
    print('------document: {0}------'.format(title[i]))
    print(document[i][:num_of_print_len], ' ... ', document[i][-num_of_print_len:])

------document: 1062890------
﻿ 年景 氣及 回顧 
 年 諸多國  ...   以不負 全體 之 殷切 寄 託 

------document: 1062836------
回顧 年 美國聯 準會 升息 三次   ...   念 本國 銀行 中 的 優質 銀行
------document: 1062891------
﻿ 壹 致股 東報 告書 
 親愛的  ...   值得 信賴 的 
 金融 服務 

------document: 1062801------
﻿ 壹 致股 東報 告書 
 張明   ...  
 
 
 穩定 
 長 總經理 

------document: 1062834------
﻿ 壹 致股 東報 告書 
 
 一  ...  爾 全球 等級 評等 
 
 
 

------document: 1062838------
一 營業 成果 
 一 國內外 金融  ...   不吝 給予 指 
 導與 鼓勵 

------document: 1062845------
各位 女士 先生 
 回顧 年 全球  ...  金融 科技 發展 開發線 投保 商機
------document: 1062849------
各位 女士 先生 
 回顧 全球 經  ...  價值 建構 永續 的 穩固 基礎 

------document: 1062880------
﻿ 一 集團 結果 
 一 國內外   ...  保險 中華 信評 穩定 
 穩定 

------document: 1062883------
﻿ 壹 致股 東報 告書 
 回顧民  ...  續 之 目標 
 最 後 敬祝 大家


### 將清理完的資料合併成表格

In [12]:
frame = pd.DataFrame({'docClass' : level, 'className' : title,
                      'docs' : document, 'file_path' : path})
frame.head()

,className,docClass,docs,file_path
0,1062890,1,﻿ 年景 氣及 回顧 \n 年 諸多國際 事件 之發 軔 及 交相 作用 對 經濟景 氣表現...,./106/1/1062890.txt
1,1062836,4,回顧 年 美國聯 準會 升息 三次 且 啟動縮 減資產 負債 表計畫 全球 經濟受 地緣 政...,./106/4/1062836.txt
2,1062891,5,﻿ 壹 致股 東報 告書 \n 親愛的 女士 先生 \n 年 全球 在 美國 貿易 保護 主...,./106/5/1062891.txt
3,1062801,6,﻿ 壹 致股 東報 告書 \n 張明 道 \n 年 全球 景氣 回升 美國 歐元區 及 等 ...,./106/6/1062801.txt
4,1062834,6,﻿ 壹 致股 東報 告書 \n \n 一 營業 結果 \n 二 營業 計畫 \n 三 未來 ...,./106/6/1062834.txt


### 將表格除存為 .pickle 格式

In [13]:
frame.to_pickle('./Output.pkl')